In [ ]:
import librosa
import numpy as np
import soundfile as sf
from scipy import signal
import sys
import random
from IPython.display import Audio, display
import os
from tqdm.notebook import tqdm
import argparse
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def load_noise_samples(noise_dir):
    """
    Load a collection of real-world noise samples from the MUSAN dataset.

    Parameters:
    -----------
    noise_dir : str
        Directory containing noise recordings

    Returns:
    --------
    dict
        Dictionary mapping noise types to audio arrays
    """
    noise_samples = {}

    print(f"Loading noise samples from {noise_dir}...")

    # Look for subdirectories
    subdirs = [d for d in os.listdir(noise_dir) if os.path.isdir(os.path.join(noise_dir, d))]

    for subdir in subdirs:
        subdir_path = os.path.join(noise_dir, subdir)
        noise_type = subdir.replace('-', '_')  # Sanitize the name

        noise_samples[noise_type] = []

        for filename in os.listdir(subdir_path):
            if filename.endswith('.wav'):
                # Load noise audio
                file_path = os.path.join(subdir_path, filename)
                try:
                    noise_audio, sr = librosa.load(file_path, sr=None)
                    noise_samples[noise_type].append((noise_audio, sr))
                    print(f"Loaded {noise_type} noise: {filename}, duration: {len(noise_audio)/sr:.2f}s")
                except Exception as e:
                    print(f"Error loading noise file {filename}: {e}")

    return noise_samples

In [ ]:
def add_basic_noise(audio, noise_factor=0.005):
    """
    Add simple Gaussian noise to an audio signal.

    Parameters:
    -----------
    audio : ndarray
        Clean audio signal
    noise_factor : float
        Amount of noise to add

    Returns:
    --------
    ndarray
        Noisy audio signal
    """
    # Generate Gaussian noise
    noise = np.random.randn(len(audio))
    augmented_audio = audio + noise_factor * noise

    # Normalize to avoid clipping
    if np.max(np.abs(augmented_audio)) > 0:
        augmented_audio = augmented_audio / np.max(np.abs(augmented_audio))

    return augmented_audio

In [ ]:
def add_realistic_noise(audio, sr, noise_samples, noise_type=None, snr_db=10):
    """
    Add realistic noise to an audio signal with a specified SNR.

    Parameters:
    -----------
    audio : ndarray
        Clean audio signal
    sr : int
        Sample rate
    noise_samples : dict
        Dictionary of noise samples
    noise_type : str, optional
        Type of noise to add (random if None)
    snr_db : float, optional
        Signal-to-noise ratio in dB (default: 10)

    Returns:
    --------
    ndarray
        Noisy audio signal
    """
    if not noise_samples:
        # Fall back to basic noise if no samples are available
        print("No noise samples available, using basic noise")
        return add_basic_noise(audio, 0.005), "basic"

    # If no noise type specified, choose randomly
    if noise_type is None:
        noise_type = random.choice(list(noise_samples.keys()))
    elif noise_type not in noise_samples:
        print(f"Warning: {noise_type} not found in noise samples. Using random noise.")
        noise_type = random.choice(list(noise_samples.keys()))

    # Choose random noise sample of specified type
    noise_audio, noise_sr = random.choice(noise_samples[noise_type])

    # Resample noise if needed
    if noise_sr != sr:
        noise_audio = librosa.resample(noise_audio, orig_sr=noise_sr, target_sr=sr)

    # If noise is shorter than audio, repeat it
    if len(noise_audio) < len(audio):
        repetitions = int(np.ceil(len(audio) / len(noise_audio)))
        noise_audio = np.tile(noise_audio, repetitions)

    # Trim or select random segment of noise to match audio length
    if len(noise_audio) > len(audio):
        start = np.random.randint(0, len(noise_audio) - len(audio) + 1)
        noise_audio = noise_audio[start:start + len(audio)]

    # Calculate scaling factor for desired SNR
    audio_power = np.mean(audio ** 2)
    noise_power = np.mean(noise_audio ** 2)

    # Convert SNR from dB to linear scale
    snr_linear = 10 ** (snr_db / 10)

    # Calculate the scaling factor
    scaling_factor = np.sqrt(audio_power / (noise_power * snr_linear))

    # Scale noise and add to audio
    scaled_noise = scaling_factor * noise_audio
    noisy_audio = audio + scaled_noise

    # Normalize to avoid clipping
    if np.max(np.abs(noisy_audio)) > 1.0:
        noisy_audio = noisy_audio / np.max(np.abs(noisy_audio))

    return noisy_audio, noise_type

In [ ]:
def apply_telephone_filter(audio, sr):
    """
    Apply a telephone-like bandpass filter (300-3400 Hz) to audio data.
    """
    # Design telephone bandpass filter (300-3400 Hz)
    nyquist = 0.5 * sr
    low = 300 / nyquist
    high = 3400 / nyquist
    b, a = signal.butter(4, [low, high], btype='band')

    # Apply filter
    filtered = signal.filtfilt(b, a, audio)

    # Normalize filtered signal
    if np.max(np.abs(filtered)) > 0:
        filtered = filtered / np.max(np.abs(filtered))

    return filtered

In [ ]:
def process_file(args):
    """
    Process a single audio file with augmentation.
    For parallel processing.
    """
    input_path, output_path, noise_samples, use_realistic_noise, snr_range = args

    try:
        # Load audio
        audio, sr = librosa.load(input_path, sr=None, mono=True)

        # Apply telephone filter
        filtered = apply_telephone_filter(audio, sr)

        if use_realistic_noise and noise_samples:
            # Apply realistic noise with random SNR
            snr_db = random.uniform(snr_range[0], snr_range[1])
            augmented, noise_type = add_realistic_noise(filtered, sr, noise_samples, snr_db=snr_db)
        else:
            # Use basic noise
            noise_factor = random.uniform(0.001, 0.01)
            augmented = add_basic_noise(filtered, noise_factor)

        # Save the result
        sf.write(output_path, augmented, sr)

        return True
    except Exception as e:
        print(f"Error processing {input_path}: {str(e)}")
        return False

In [ ]:
def process_directory(input_dir, output_dir, noise_dir=None, snr_range=(5, 25), use_parallel=True, n_processes=None):
    """
    Process all WAV files in a directory structure with realistic noise.

    Parameters:
    -----------
    input_dir : str
        Input directory containing clean WAV files
    output_dir : str
        Output directory for processed WAV files
    noise_dir : str, optional
        Directory containing noise samples
    snr_range : tuple, optional
        Range of SNR values in dB (default: 5-25 dB)
    use_parallel : bool
        Whether to use parallel processing
    n_processes : int, optional
        Number of processes to use (default: CPU count - 1)
    """
    # Load noise samples if a noise directory is provided
    noise_samples = {}
    use_realistic_noise = False

    if noise_dir and os.path.exists(noise_dir):
        noise_samples = load_noise_samples(noise_dir)
        if noise_samples:
            use_realistic_noise = True
            print(f"Loaded {sum(len(samples) for samples in noise_samples.values())} noise samples of {len(noise_samples)} types")

    # Count files
    print("Counting WAV files (this may take a while for large datasets)...")
    total_files = sum(1 for root, _, files in os.walk(input_dir)
                     for file in files if file.lower().endswith('.wav'))

    print(f"Found {total_files} WAV files to process")

    if use_parallel:
        # Process files in parallel
        from multiprocessing import Pool, cpu_count

        if n_processes is None:
            n_processes = max(1, cpu_count() - 1)

        print(f"Using {n_processes} CPU cores for processing")

        # Collect tasks
        tasks = []
        for root, _, files in os.walk(input_dir):
            for file in files:
                if file.lower().endswith('.wav'):
                    input_path = os.path.join(root, file)
                    rel_path = os.path.relpath(input_path, input_dir)
                    output_path = os.path.join(output_dir, rel_path)
                    os.makedirs(os.path.dirname(output_path), exist_ok=True)
                    tasks.append((input_path, output_path, noise_samples, use_realistic_noise, snr_range))

        # Process files in parallel
        start_time = time.time()
        with Pool(n_processes) as pool:
            results = list(tqdm(pool.imap(process_file, tasks), total=total_files, desc="Processing files"))

        # Report results
        processed_files = sum(results)
        failed_files = total_files - processed_files

    else:
        # Sequential processing
        processed_files = 0
        failed_files = 0
        start_time = time.time()

        # Create progress bar
        pbar = tqdm(total=total_files, desc="Processing files")

        # Process each file
        for root, _, files in os.walk(input_dir):
            for file in files:
                if file.lower().endswith('.wav'):
                    input_path = os.path.join(root, file)

                    # Create corresponding output path
                    rel_path = os.path.relpath(input_path, input_dir)
                    output_path = os.path.join(output_dir, rel_path)

                    # Create output directory if needed
                    os.makedirs(os.path.dirname(output_path), exist_ok=True)

                    try:
                        # Load audio
                        audio, sr = librosa.load(input_path, sr=None)

                        # Apply telephone filter
                        filtered = apply_telephone_filter(audio, sr)

                        if use_realistic_noise:
                            # Apply realistic noise with random SNR
                            snr_db = random.uniform(snr_range[0], snr_range[1])
                            augmented, noise_type = add_realistic_noise(filtered, sr, noise_samples, snr_db=snr_db)
                        else:
                            # Use basic noise
                            noise_factor = random.uniform(0.001, 0.01)
                            augmented = add_basic_noise(filtered, noise_factor)

                        # Save processed audio
                        sf.write(output_path, augmented, sr)

                        processed_files += 1

                    except Exception as e:
                        print(f"Error processing {input_path}: {e}")
                        failed_files += 1

                    pbar.update(1)

        pbar.close()

    # Final report
    total_time = time.time() - start_time
    print(f"\nProcessing complete in {total_time/60:.1f} minutes.")
    print(f"Successfully processed {processed_files} files.")
    if failed_files > 0:
        print(f"Failed to process {failed_files} files.")

In [ ]:
def demonstrate_noise_effects(input_file, output_dir, noise_dir=None):
    """
    Create examples of the same audio file with different noise conditions.
    This helps with assessing the realism of the augmentation.

    Parameters:
    -----------
    input_file : str
        Path to input audio file
    output_dir : str
        Directory to save output files
    noise_dir : str, optional
        Directory containing noise samples
    """
    # Load input file
    audio, sr = librosa.load(input_file, sr=None)

    # Apply telephone filter to original
    filtered_audio = apply_telephone_filter(audio, sr)

    # Save filtered version (no noise)
    os.makedirs(output_dir, exist_ok=True)
    base_filename = os.path.splitext(os.path.basename(input_file))[0]
    telephone_path = os.path.join(output_dir, f"{base_filename}_telephone.wav")
    sf.write(telephone_path, filtered_audio, sr)

    # Also save original for comparison
    original_path = os.path.join(output_dir, f"{base_filename}_original.wav")
    sf.write(original_path, audio, sr)

    # Generate examples with basic noise
    for noise_factor in [0.001, 0.005, 0.01, 0.02]:
        # Apply basic noise
        noisy_audio = add_basic_noise(filtered_audio, noise_factor)

        # Save example
        output_path = os.path.join(
            output_dir,
            f"{base_filename}_telephone_basicnoise_{noise_factor:.3f}.wav"
        )
        sf.write(output_path, noisy_audio, sr)
        print(f"Created basic noise example with factor {noise_factor:.3f}")

    # Load noise samples if a noise directory is provided
    if noise_dir and os.path.exists(noise_dir):
        noise_samples = load_noise_samples(noise_dir)

        # Generate examples with different noise types and SNRs
        for noise_type in noise_samples.keys():
            for snr_db in [5, 10, 15, 20]:
                # Apply noise
                noisy_audio, _ = add_realistic_noise(
                    filtered_audio, sr, noise_samples,
                    noise_type=noise_type, snr_db=snr_db
                )

                # Save example
                output_path = os.path.join(
                    output_dir,
                    f"{base_filename}_telephone_{noise_type}_SNR{snr_db}.wav"
                )
                sf.write(output_path, noisy_audio, sr)
                print(f"Created {noise_type} noise example with SNR {snr_db}dB")

In [ ]:
# Main execution
if __name__ == "__main__":
    # Define paths
    INPUT_DIR = "/content/drive/Shareddrives/VoxCeleb1/Dev"
    OUTPUT_DIR = "/content/drive/Shareddrives/VoxCeleb1/Dev_Augmented"
    NOISE_DIR = "/content/drive/Shareddrives/286 2/MUSAN/extracted/musan/noise/"

    # Print paths for confirmation
    print(f"Input directory: {INPUT_DIR}")
    print(f"Output directory: {OUTPUT_DIR}")
    print(f"Noise samples directory: {NOISE_DIR}")

    # Ask for mode
    mode = input("Choose mode (demo/process): ")

    if mode.lower() == "demo":
        # Demonstrate noise effects on a single file
        input_file = input("Enter path to a WAV file to test: ")
        demonstrate_noise_effects(input_file, OUTPUT_DIR, NOISE_DIR)
    elif mode.lower() == "process":
        # Process all files
        use_parallel = input("Use parallel processing? (y/n): ").lower() == 'y'
        process_directory(INPUT_DIR, OUTPUT_DIR, NOISE_DIR, use_parallel=use_parallel)
    else:
        print("Invalid mode. Exiting.")

Input directory: /content/drive/Shareddrives/VoxCeleb1/Dev
Output directory: /content/drive/Shareddrives/VoxCeleb1/Dev_Augmented
Noise samples directory: /content/drive/Shareddrives/286 2/MUSAN/extracted/musan/noise/
Loading noise samples from /content/drive/Shareddrives/286 2/MUSAN/extracted/musan/noise/...
Loaded sound_bible noise: noise-sound-bible-0006.wav, duration: 0.77s
Loaded sound_bible noise: noise-sound-bible-0067.wav, duration: 27.74s
Loaded sound_bible noise: noise-sound-bible-0056.wav, duration: 7.89s
Loaded sound_bible noise: noise-sound-bible-0040.wav, duration: 9.14s
Loaded sound_bible noise: noise-sound-bible-0009.wav, duration: 2.12s
Loaded sound_bible noise: noise-sound-bible-0019.wav, duration: 6.92s
Loaded sound_bible noise: noise-sound-bible-0077.wav, duration: 5.15s
Loaded sound_bible noise: noise-sound-bible-0080.wav, duration: 4.70s
Loaded sound_bible noise: noise-sound-bible-0055.wav, duration: 18.10s
Loaded sound_bible noise: noise-sound-bible-0082.wav, dura

Processing files:   0%|          | 0/10193 [00:00<?, ?it/s]


Processing complete in 533.7 minutes.
Successfully processed 10193 files.


# preview of wav before augmentation

In [ ]:
input_path = '/content/drive/Shareddrives/CS 286 Project Data/extracted_vox1_dev/wav/id10001/1zcIwhmdeo4/00001.wav'
noise_factor=0.005
audio, sr = librosa.load(input_path, sr=None)
display(Audio(audio, rate=sr))

# add random noise at random factor + preview sample

In [ ]:

input_path = '/content/drive/Shareddrives/CS 286 Project Data/extracted_vox1_dev/wav/id10001/1zcIwhmdeo4/00001.wav'
noise_factor_int=np.random.randint(25, 50)
noise_factor = noise_factor_int / 1000
print(noise_factor)
audio, sr = librosa.load(input_path, sr=None)
noisy_audio = add_noise(audio, noise_factor)
display(Audio(noisy_audio, rate=sr))

0.043


# apply telephone filter + preview sample

In [ ]:
def apply_telephone_filter(y, sr):

  nyquist = 0.5 * sr
  low = 300 / nyquist
  high = 3400 / nyquist
  b, a = signal.butter(4, [low, high], btype='band')

  # Apply filter
  filtered = signal.filtfilt(b, a, audio)

  # Normalize filtered signal
  if np.max(np.abs(filtered)) > 0:
    filtered = filtered / np.max(np.abs(filtered))

  return filtered

# telephone_filter_base = apply_telephone_filter(audio, sr)
# telephone_and_noise = apply_telephone_filter(noisy_audio, sr)
# print("just telephone:")
# display(Audio(telephone_filter_base, rate=sr))
# print("both:")
# display(Audio(telephone_and_noise, rate=sr))

In [ ]:
def augment(input_path, output_path):
  audio, sr = librosa.load(input_path, sr=None)
  noisy_audio = add_noise(audio, noise_factor)
  telephone_and_noise = apply_telephone_filter(noisy_audio, sr)
  sf.write(output_path, telephone_and_noise, sr)

In [ ]:
def process_directory(input_dir, output_dir):
    """
    Process all WAV files in a directory structure, applying the telephone filter
    to each one while preserving the relative paths.

    Parameters:
    -----------
    input_dir : str
        Root directory containing WAV files to process
    output_dir : str
        Root directory where processed WAV files will be saved
    """
    print("Counting WAV files (this may take a while for large datasets)...")
    total_files = 0
    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.lower().endswith('.wav'):
                total_files += 1

    print(f"Found {total_files} WAV files to process.")

    # Process each WAV file
    processed_files = 0
    failed_files = 0
    start_time = time.time()
    # Create progress bar
    pbar = tqdm(total=total_files, desc="Processing files")

    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.lower().endswith('.wav'):
                input_path = os.path.join(root, file)

                # Create corresponding output path with same subdirectory structure
                rel_path = os.path.relpath(input_path, input_dir)
                output_path = os.path.join(output_dir, rel_path)

                # Create output directory if it doesn't exist
                os.makedirs(os.path.dirname(output_path), exist_ok=True)

                try:
                    # Load audio
                    audio, sr = librosa.load(input_path, sr=None)

                    # Apply filter
                    filtered = apply_telephone_filter(audio, sr)

                    # Save filtered audio
                    sf.write(output_path, filtered, sr)

                    processed_files += 1
                except Exception as e:
                    print(f"Error processing {input_path}: {str(e)}")
                    failed_files += 1

                pbar.update(1)

                # Display occasional status updates
                if processed_files % 100 == 0 and processed_files > 0:
                    elapsed = time.time() - start_time
                    files_per_sec = processed_files / elapsed
                    estimated_total = elapsed * (total_files / processed_files)
                    estimated_remaining = estimated_total - elapsed

                    print(f"\nProcessed {processed_files}/{total_files} files " +
                          f"({processed_files/total_files*100:.1f}%)")
                    print(f"Speed: {files_per_sec:.2f} files/sec")
                    print(f"Elapsed: {elapsed/60:.1f} minutes")
                    print(f"Estimated time remaining: {estimated_remaining/60:.1f} minutes")

    pbar.close()

    # Final report
    total_time = time.time() - start_time
    print(f"\nProcessing complete in {total_time/60:.1f} minutes.")
    print(f"Successfully processed {processed_files} files.")
    if failed_files > 0:
        print(f"Failed to process {failed_files} files. Check console output for details.")

In [ ]:
from multiprocessing import Pool, cpu_count

def process_file(args):
    input_path, output_path, noise_factor = args
    try:
        # Load audio
        audio, sr = librosa.load(input_path, sr=None)

        # Apply telephone filter
        filtered = apply_telephone_filter(audio, sr)

        # Add noise
        augmented = add_noise(filtered, noise_factor)

        # Save filtered audio
        sf.write(output_path, augmented, sr)

        return True
    except Exception as e:
        print(f"Error processing {input_path}: {str(e)}")
        return False

def process_directory_parallel(input_dir, output_dir, noise_factor=0.005, n_processes=None):
    '''Process directory using multiple CPU cores'''
    if n_processes is None:
        n_processes = cpu_count() - 1 or 1

    print(f"Using {n_processes} CPU cores for processing")

    # Collect all files and their output paths
    tasks = []
    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.lower().endswith('.wav'):
                input_path = os.path.join(root, file)
                rel_path = os.path.relpath(input_path, input_dir)
                output_path = os.path.join(output_dir, rel_path)
                os.makedirs(os.path.dirname(output_path), exist_ok=True)
                tasks.append((input_path, output_path, noise_factor))

    total_files = len(tasks)
    print(f"Found {total_files} WAV files to process.")

    # Process files in parallel
    start_time = time.time()
    with Pool(n_processes) as pool:
        results = list(tqdm(pool.imap(process_file, tasks), total=total_files))

    # Report results
    processed_files = sum(results)
    failed_files = total_files - processed_files

    total_time = time.time() - start_time
    print(f"\nParallel processing complete in {total_time/60:.1f} minutes.")
    print(f"Successfully processed {processed_files} files.")
    if failed_files > 0:
        print(f"Failed to process {failed_files} files.")

In [ ]:
input_dir = "/content/drive/Shareddrives/VoxCeleb1/Dev"
output_dir = "/content/drive/Shareddrives/VoxCeleb1/Dev_Augmented"
process_directory_parallel(input_dir=input_dir, output_dir=output_dir)

Using 11 CPU cores for processing


KeyboardInterrupt: 

In [1]:
import os
import numpy as np
import time
from scipy import signal
import librosa
import soundfile as sf
from tqdm.notebook import tqdm
import warnings

# Suppress specific warnings that might be occurring
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

def add_noise(audio, noise_factor=0.005):
    """
    Add Gaussian noise to an audio signal.
    """
    # Validate input
    if audio is None or len(audio) == 0:
        raise ValueError("Empty or None audio data provided to add_noise")

    # Generate Gaussian noise and add it to the audio signal
    noise = np.random.randn(len(audio))
    augmented_audio = audio + noise_factor * noise

    # Normalize to avoid clipping
    if np.max(np.abs(augmented_audio)) > 0:
        augmented_audio = augmented_audio / np.max(np.abs(augmented_audio))

    return augmented_audio

def apply_telephone_filter(audio, sr):
    """
    Apply a telephone-like bandpass filter (300-3400 Hz) to audio data.
    """
    # Validate input
    if audio is None or len(audio) == 0:
        raise ValueError("Empty or None audio data provided to apply_telephone_filter")

    # Design telephone bandpass filter (300-3400 Hz)
    nyquist = 0.5 * sr
    low = 300 / nyquist
    high = 3400 / nyquist

    # Ensure filter parameters are valid
    if low >= 1.0 or high >= 1.0:
        raise ValueError(f"Invalid filter parameters: low={low}, high={high} (must be < 1.0)")

    b, a = signal.butter(4, [low, high], btype='band')

    # Apply filter
    filtered = signal.filtfilt(b, a, audio)

    # Normalize filtered signal
    if np.max(np.abs(filtered)) > 0:
        filtered = filtered / np.max(np.abs(filtered))

    return filtered

def process_directory(input_dir, output_dir, noise_factor=0.005):
    """
    Process all WAV files in a directory structure.
    """
    print("Counting WAV files (this may take a while for large datasets)...")
    total_files = 0
    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.lower().endswith('.wav'):
                total_files += 1

    print(f"Found {total_files} WAV files to process.")

    # Process each WAV file
    processed_files = 0
    failed_files = 0
    start_time = time.time()

    # Create progress bar
    pbar = tqdm(total=total_files, desc="Processing files")

    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.lower().endswith('.wav'):
                input_path = os.path.join(root, file)

                # Create corresponding output path with same subdirectory structure
                rel_path = os.path.relpath(input_path, input_dir)
                output_path = os.path.join(output_dir, rel_path)

                # Create output directory if it doesn't exist
                os.makedirs(os.path.dirname(output_path), exist_ok=True)

                try:
                    # Log file processing attempt
                    print(f"Processing file: {input_path}")

                    # Check if file exists and has size > 0
                    if not os.path.exists(input_path):
                        raise FileNotFoundError(f"File does not exist: {input_path}")

                    if os.path.getsize(input_path) == 0:
                        raise ValueError(f"File is empty: {input_path}")

                    # Load audio with explicit error handling
                    try:
                        audio, sr = librosa.load(input_path, sr=None, mono=True)
                        print(f"Successfully loaded audio: shape={audio.shape}, sr={sr}")
                    except Exception as e:
                        raise Exception(f"Failed to load audio: {str(e)}")

                    # Verify the audio data
                    if audio is None or len(audio) == 0:
                        raise ValueError(f"Empty audio data loaded from {input_path}")

                    # Apply telephone filter
                    try:
                        filtered = apply_telephone_filter(audio, sr)
                        print(f"Successfully applied filter: shape={filtered.shape}")
                    except Exception as e:
                        raise Exception(f"Failed to apply filter: {str(e)}")

                    # Add noise
                    try:
                        augmented = add_noise(filtered, noise_factor)
                        print(f"Successfully added noise: shape={augmented.shape}")
                    except Exception as e:
                        raise Exception(f"Failed to add noise: {str(e)}")

                    # Save filtered audio
                    try:
                        sf.write(output_path, augmented, sr)
                        print(f"Successfully saved to: {output_path}")
                    except Exception as e:
                        raise Exception(f"Failed to save audio: {str(e)}")

                    processed_files += 1
                    print(f"Successfully processed: {input_path}")

                except Exception as e:
                    failed_files += 1
                    print(f"Error processing {input_path}: {str(e)}")

                pbar.update(1)

                # Display occasional status updates
                if processed_files % 10 == 0 and processed_files > 0:
                    elapsed = time.time() - start_time
                    files_per_sec = processed_files / elapsed if elapsed > 0 else 0
                    estimated_total = elapsed * (total_files / processed_files) if processed_files > 0 else 0
                    estimated_remaining = estimated_total - elapsed

                    print(f"\nProcessed {processed_files}/{total_files} files " +
                          f"({processed_files/total_files*100:.1f}%)")
                    print(f"Speed: {files_per_sec:.2f} files/sec")
                    print(f"Elapsed: {elapsed/60:.1f} minutes")
                    print(f"Estimated time remaining: {estimated_remaining/60:.1f} minutes")
                    print(f"Failed files so far: {failed_files}")

    pbar.close()

    # Final report
    total_time = time.time() - start_time
    print(f"\nProcessing complete in {total_time/60:.1f} minutes.")
    print(f"Successfully processed {processed_files} files.")
    if failed_files > 0:
        print(f"Failed to process {failed_files} files. Check console output for details.")

# Test a single file with detailed error reporting
def test_file(input_path):
    """
    Test loading and processing a single file with detailed error reporting.
    """
    print(f"\nTesting file: {input_path}")

    try:
        print("Checking if file exists...")
        if not os.path.exists(input_path):
            raise FileNotFoundError(f"File does not exist: {input_path}")

        print(f"File size: {os.path.getsize(input_path)} bytes")

        print("Attempting to load audio...")
        audio, sr = librosa.load(input_path, sr=None, mono=True)

        print(f"Successfully loaded audio:")
        print(f"  Shape: {audio.shape}")
        print(f"  Sample rate: {sr} Hz")
        print(f"  Duration: {len(audio)/sr:.2f} seconds")
        print(f"  Range: [{np.min(audio):.2f}, {np.max(audio):.2f}]")

        print("\nAttempting to apply telephone filter...")
        filtered = apply_telephone_filter(audio, sr)
        print("Filter applied successfully")

        print("\nAttempting to add noise...")
        augmented = add_noise(filtered, 0.005)
        print("Noise added successfully")

        print("\nFile processed successfully!")
        return True

    except Exception as e:
        print(f"\nError processing file: {str(e)}")
        return False

# Attempt to fix a common issue with loading WAV files
def fix_wav_file(input_path, output_path):
    """
    Attempt to fix corrupted WAV files by re-encoding them.
    """
    try:
        # Try loading with soundfile first (faster)
        try:
            data, sr = sf.read(input_path)
            sf.write(output_path, data, sr)
            return True
        except:
            pass

        # If that fails, try librosa with more lenient loading
        try:
            y, sr = librosa.load(input_path, sr=None, mono=True, res_type='kaiser_fast')
            sf.write(output_path, y, sr)
            return True
        except:
            pass

        # If still fails, try raw reading and converting
        try:
            import wave
            with wave.open(input_path, 'rb') as wf:
                frames = wf.readframes(wf.getnframes())
                rate = wf.getframerate()
                channels = wf.getnchannels()
                width = wf.getsampwidth()

            with wave.open(output_path, 'wb') as wf:
                wf.setnchannels(channels)
                wf.setsampwidth(width)
                wf.setframerate(rate)
                wf.writeframes(frames)
            return True
        except:
            return False
    except Exception as e:
        print(f"Error fixing WAV file: {str(e)}")
        return False

# Set your input and output directories
INPUT_DIR = "/content/drive/Shareddrives/VoxCeleb1/Test/wav"
OUTPUT_DIR = "/content/drive/Shareddrives/VoxCeleb1/Test_Augmented/wav"

# Confirm directories before processing
print(f"Input directory: {INPUT_DIR}")
print(f"Output directory: {OUTPUT_DIR}")

# Ask for confirmation
confirmation = input("Do you want to test a single file (t), process all files (p), or fix WAV files (f): ")

if confirmation.lower() == 't':
    # Test a single file
    file_path = input("Enter path to a WAV file to test: ")
    test_file(file_path)
elif confirmation.lower() == 'p':
    # Process all files
    process_directory(INPUT_DIR, OUTPUT_DIR)
elif confirmation.lower() == 'f':
    # Fix WAV files
    input_path = input("Enter path to WAV file to fix: ")
    output_path = input("Enter path to save fixed WAV file: ")
    if fix_wav_file(input_path, output_path):
        print("File fixed successfully!")
    else:
        print("Failed to fix file.")
else:
    print("Operation cancelled.")

Input directory: /content/drive/Shareddrives/VoxCeleb1/Test/wav
Output directory: /content/drive/Shareddrives/VoxCeleb1/Test_Augmented/wav
Do you want to test a single file (t), process all files (p), or fix WAV files (f): p
Counting WAV files (this may take a while for large datasets)...
Found 1519 WAV files to process.


Processing files:   0%|          | 0/1519 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
Successfully loaded audio: shape=(140161,), sr=16000
Successfully applied filter: shape=(140161,)
Successfully added noise: shape=(140161,)
Successfully saved to: /content/drive/Shareddrives/VoxCeleb1/Test_Augmented/wav/wav/id10279/4Q1IvdayPR8/00013.wav
Successfully processed: /content/drive/Shareddrives/VoxCeleb1/Test/wav/wav/id10279/4Q1IvdayPR8/00013.wav
Processing file: /content/drive/Shareddrives/VoxCeleb1/Test/wav/wav/id10279/4Q1IvdayPR8/00014.wav
Successfully loaded audio: shape=(73601,), sr=16000
Successfully applied filter: shape=(73601,)
Successfully added noise: shape=(73601,)
Successfully saved to: /content/drive/Shareddrives/VoxCeleb1/Test_Augmented/wav/wav/id10279/4Q1IvdayPR8/00014.wav
Successfully processed: /content/drive/Shareddrives/VoxCeleb1/Test/wav/wav/id10279/4Q1IvdayPR8/00014.wav
Processing file: /content/drive/Shareddrives/VoxCeleb1/Test/wav/wav/id10279/4Q1IvdayPR8/00012.wav
Successfully loaded audio: shape=(1094